# Neural Network
- *FFNN* : feed forward neural network (only execute forward propagation)
- Chapter5 -p301 explanation of theory (difficult...)

In [1]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

root = "./data"

transform = transforms.Compose(
    [transforms.ToTensor(), #Tensorオブジェクトに変換
    transforms.Normalize((0.5), (0.5)), #平均0.5, 標準偏差0.5で正規化
    lambda x: x.view(-1),] #データの形状を(28,28)から(784,)に変換
)

f_mnist_train = datasets.FashionMNIST(
    root=root,
    download=True,
    train=True,
    transform=transform,
)

f_mnist_test = datasets.FashionMNIST(
    root=root,
    download=True,
    train=False,
    transform=transform,
)
batch_size = 64

train_dataloader = DataLoader(
    f_mnist_train,
    batch_size=batch_size,
    shuffle=True,
)

test_dataloader = DataLoader(
    f_mnist_test,
    batch_size=batch_size,
    shuffle=False,
)

for (x,t) in train_dataloader:
    print(x.shape)
    print(t.shape)
    break

for (x,t) in test_dataloader:
    print(x.shape)
    print(t.shape)
    break

100.0%


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100.6%


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100.0%


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



119.3%

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

torch.Size([64, 784])
torch.Size([64])
torch.Size([64, 784])
torch.Size([64])
